In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import cv2
import json

import sys
sys.path.append('..')

from shared import metrics
from shared.data import KITTIData,  VisualOdometry, draw_matches, draw_keypoints
from shared import common

%matplotlib widget
import matplotlib.pyplot as plt

from concurrent.futures import ProcessPoolExecutor

import time
timestr = time.strftime("%Y%m%d-%H%M%S")

RESULTS_DIRNAME = f'stereo_results_{timestr}'
RESULTS_DIRPATH = os.path.join('results', RESULTS_DIRNAME)
os.makedirs(RESULTS_DIRPATH, exist_ok=True)

DATASET_DIR = os.path.join('../', 'data/KITTI/dataset')

In [ ]:
seuquences = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10']

for seq_id in seuquences:
    seq_dirpath = os.path.join(RESULTS_DIRPATH, seq_id)
    
    poses_fpath = os.path.join(seq_dirpath, 'poses.npz')
    metrics_fpath = os.path.join(seq_dirpath, 'metrics.json')
    
    dataset = KITTIData(DATASET_DIR, sequence_id=seq_id)
    gt_poses = dataset.get_poses()
    Q_left = dataset.get_left_Q_matrix()
    C_left, _ = dataset.get_С_matrix()

    vo = VisualOdometry()

    def process_transform(idx):
        c_pose = gt_poses[idx]
        n_pose = gt_poses[idx+1]

        c_l_img, c_r_img = dataset.get_images(idx)
        n_l_img, n_r_img = dataset.get_images(idx+1)

        c_depth_frame = vo.process_depth(c_l_img, c_r_img, Q_left)
        n_depth_frame = vo.process_depth(n_l_img, n_r_img, Q_left)

        c_feats, n_feats = vo.get_features(c_l_img, n_l_img)
        c_pnts_3d, c_ft_idxs = vo.reproject_2d_to_3d_points(c_feats, c_depth_frame)
        n_pnts_3d, n_ft_idxs = vo.reproject_2d_to_3d_points(n_feats, n_depth_frame)

        ft_idxs = c_ft_idxs & n_ft_idxs

        c_pnts_3d = c_pnts_3d[ft_idxs]
        n_pnts_3d = n_pnts_3d[ft_idxs]
        c_feats = c_feats[ft_idxs]
        n_feats = n_feats[ft_idxs]

        transform = vo.get_transform(c_feats, n_feats, c_pnts_3d, n_pnts_3d, C_left, type_='PnPRansac')

        n_pred_pose = vo.get_next_pose(transform, c_pose)
        err = np.abs(n_pred_pose-n_pose)
        if np.any(err > 1):
            raise Exception(f'>>> Failed transform: {idx}')
        return transform
    
    poses_count = len(gt_poses)-1
    try:
        with ProcessPoolExecutor(8) as ex:
            transforms = ex.map(process_transform, range(poses_count))
            
        poses = [
            np.eye(4)
        ]

        for transform in transforms:
            n_pose = vo.get_next_pose(transform, poses[-1])
            poses.append(n_pose)

        poses = np.array(poses)
        metrics_ate = metrics.compute_ATE(poses, gt_poses[:len(poses)])
        rpe_trns, rpe_rot = metrics.compute_RPE(poses, gt_poses[:len(poses)])
        
        os.makedirs(seq_dirpath, exist_ok=True)

        np.savez(poses_fpath, poses=poses)
        metrics_data = {
            'ATE': metrics_ate,
            'RPE_R': rpe_rot,
            'RPE_T': rpe_trns
        }
        
        with open(metrics_fpath, 'w') as f:
            json.dump(metrics_data, f)
        
    except Exception as e:
        print(f'Sequence {seq_id} failed: {e}')
